# Imports

## Installs

In [1]:

#!pip install layoutparser torchvision
#!pip install layoutparser[ocr]
#!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
#!apt-get update
#!apt-get install -y tesseract-ocr
#!pip install Pillow==9.5.0

#!pip install pytesseract


!pip install transformers

!pip install -q --upgrade transformers accelerate optimum
!pip install -q --no-build-isolation auto-gptq



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Imports



In [1]:
import pandas as pd
import numpy as np
import time
import json
import torch
print(torch.__version__)  # Should print a compatible version
print(torch.cuda.is_available())  #


import glob
import os
import re
import random
import cv2
import PIL
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline



from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM
from transformers import pipeline
from huggingface_hub import login

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import accelerate 
from accelerate import Accelerator






2.5.1+cu124
True


/ext3/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/ext3/miniforge3/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/ext3/miniforge3/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/ext3/miniforge3/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(ca

## Loading Data

In [2]:
df = pd.read_csv("../Data/opec.csv")

In [3]:
df

,date,text
0,2021-12-14,Communique of the African Petroleum Producers’...
1,2021-12-10,The Declaration of Cooperation of OPEC and non...
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...
3,2021-12-01,Opening address to the 182nd Meeting of the OP...
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...
...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...
128,2024-03-03,Several OPEC+ countries announce extension of ...
129,2024-02-19,OPEC and Africa exchange views on array of key...
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...


# Model Download and Initializaton

## Login Token

In [4]:
login(token ='')

## Llama 8b Instruct

In [6]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.device

Loading checkpoint shards: 100%|██████████| 4/4 [00:51<00:00, 12.86s/it]


device(type='cuda', index=0)

In [9]:
model.device

device(type='cuda', index=0)

## Mistral NeMo

In [5]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)



Loading checkpoint shards: 100%|██████████| 5/5 [00:19<00:00,  3.91s/it]


In [11]:
model.device

device(type='cuda', index=0)

## Prompts

### Llama Prompting

In [7]:
# Define extraction function for borrowers
def llama_sentiment(text):
    # Simplified prompt for extracting only borrowers
    prompt = f"""

      <|begin_of_text|><|start_header_id|>system<|end_header_id|>
      
      You are an oil and OPEC annoucement sentiment analyzer. You will be given a document with oil and OPEC news. You have 
      to give a sentiment score to that document based on what it means for the oil market. 

      Give a continuous score between -1.0 and 1.0 Give it up to one decimal place. 
      
        ** -1 being the worst news for oil price and sales. 
        ** -1 to 0 being bad news for oil price and sales. 
        ** 0 being neutral news for oil price and sales. 
        ** 0 to 1 being good news for oil price and sales. 
        ** 1 being exremely good news for oil price and sales. 


      Please present your output as a float. 

      Output Example for your reference:
      
      JSON: {{"sentiment_score": "0.2"}}
      
      <|eot_id|><|start_header_id|>user<|end_header_id|>

        
      [INST]
      This is the OPEC annoucement you have to analyze and output the sentiment for. 

      {text} <|eot_id|>

      <|start_header_id|>assistant<|end_header_id|>

      JSON: 
      
    """

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    inputs = inputs.input_ids
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Use max_new_tokens to control the output
    outputs = model.generate(inputs, max_new_tokens=512, temperature=0.2)
    extracted_info = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    # Clean up the string by removing unnecessary escape characters
    #cleaned_output = extracted_info.replace('\\n', '').replace('\\\'', '\'').replace('\'', '"')

    marker = "JSON:"
    start = extracted_info.rfind(marker)
    if start != -1:
        json_output = extracted_info[start + len(marker):].strip() 
        try: 
            extracted_json = json.loads(json_output)
            return extracted_json
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed: {e}. Raw JSON output was: {json_output}")
            return json_output
            
        
    else:
        return None
    

  

### Mistral Prompting

In [6]:
# Define extraction function for borrowers
def mistral_sentiment(text):
    # Simplified prompt for extracting only borrowers
    prompt = f"""

      <s>[INST]
      
      You are an oil and OPEC annoucement sentiment analyzer. You will be given a document with oil and OPEC news. You have 
      to give a sentiment score to that document based on what it means for the oil market. 

      Give a continuous score between -1.0 and 1.0 Give it up to one decimal place. 
      
        ** -1 being the worst news for oil price and sales. 
        ** -1 to 0 being bad news for oil price and sales. 
        ** 0 being neutral news for oil price and sales. 
        ** 0 to 1 being good news for oil price and sales. 
        ** 1 being exremely good news for oil price and sales. 


      Please present your output as a float. 

      Output Example for your reference: [/INST]
      
      JSON: {{"sentiment_score": "0.2"}}
      
      </s>

        
      [INST]
      This is the OPEC annoucement you have to analyze and output the sentiment for. 

      {text} [/INST]

      [INST]

      Now give a sentiment score. Please return only the JSON object. Start output after saying "JSON:" </s>

      [/INST]

      
      
    """

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    inputs = inputs.input_ids
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Use max_new_tokens to control the output
    outputs = model.generate(inputs, max_new_tokens=512, temperature=0.1)
    extracted_info = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    # Clean up the string by removing unnecessary escape characters
    #cleaned_output = extracted_info.replace('\\n', '').replace('\\\'', '\'').replace('\'', '"')

    marker = "JSON:"
    start = extracted_info.rfind(marker)
    if start != -1:
        json_output = extracted_info[start + len(marker):].strip() 
        try: 
            extracted_json = json.loads(json_output)
            return extracted_json
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed: {e}. Raw JSON output was: {json_output}")
            return json_output
            
        
    else:
        return None
    

  

# Runs

## Manual

In [31]:
outs = []

In [8]:
text1 = df['text'][0]
text2 = df['text'][1]

In [9]:
text1

'Communique of the African Petroleum Producers’ Organization (APPO)\nNo --\nAlgiers, Algeria\n14 Dec 2021\nCommunique of the African Petroleum Producers’ Organization (APPO).\n41st Ordinary Session of the Council of Ministers.\nCouncil of Ministers of APPO on 14 December 2021 in Algiers, Algeria.\nThe Council of Ministers of the African Petroleum Producers’ Organization (APPO) just ended its 41st Ministerial Session with a resolve to continue the exploitation of its Member Countries’ huge oil and gas resources for the economic emancipation of its people while also exploring the use of renewable energies.\nThe Session which was presided by the President of the Organization, His Excellency Mohamed Arkab, Minister of Mines and Energy of the People’s Democratic Republic of Algeria, was attended by the Ministers and Heads of Delegation of the 14 Member Countries of the Organization, namely Algeria, Angola, Benin, Cameroun, Chad, Congo, Democratic Republic of Congo, Egypt, Equatorial Guinea,

In [12]:
out = mistral_sentiment(text2)



/ext3/miniforge3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [13]:
out

{'sentiment_score': '0.5'}

## Automated

In [14]:
texts = df['text']

In [15]:
sentiments = []

for i, text in enumerate(texts): 
    sentiment = mistral_sentiment(text)
    sentiments.append(sentiment)
    print(i)

/ext3/miniforge3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

128


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


129


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


130
131


In [16]:
df_mistral = pd.DataFrame(sentiments)
df_mistral

,sentiment_score
0,0.0
1,0.5
2,0.5
3,0.4
4,0.5
...,...
127,0.5
128,-0.5
129,0.0
130,0.5


# Outputs

In [78]:
#df['sentiment_mistral'] = sentiment_2_df['sentiment_score']

,date,text,sentiment_mistral
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5
...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5


In [80]:
os.getcwd()

'/scratch/us2055/Oil_Price/Notebooks'

In [81]:
df.to_csv('../Outputs/sentiment_mistral.csv')

## Appending Sentiment

In [21]:
df_sentiment_combined = pd.read_csv("../Outputs/sentiment_combined.csv")
df_sentiment_combined 

,Unnamed: 0.1,Unnamed: 0,date,text,sentiment_mistral,sentiment_llama
0,0,0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1
1,1,1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0
2,2,2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0
3,3,3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0
4,4,4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0
...,...,...,...,...,...,...
127,127,127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5
128,128,128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6
129,129,129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5
130,130,130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0


In [25]:
df_sentiment_combined['sentiment_mistral_1'] = df_mistral['sentiment_score']
df_sentiment_combined = df_sentiment_combined.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
df_sentiment_combined

,date,text,sentiment_mistral,sentiment_llama,sentiment_mistral_1
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0,0.5
...,...,...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6,-0.5
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5,0.0
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0,0.5


In [28]:
df_sentiment_combined.to_csv('../Outputs/sentiment_combined_1.csv', index = False)

In [29]:
df_sentiment_combined

,date,text,sentiment_mistral,sentiment_llama,sentiment_mistral_1
0,2021-12-14,Communique of the African Petroleum Producers’...,0.0,0.1,0.0
1,2021-12-10,The Declaration of Cooperation of OPEC and non...,0.5,0.0,0.5
2,2021-12-02,23rd OPEC and non-OPEC Ministerial Meeting\nNo...,0.5,0.0,0.5
3,2021-12-01,Opening address to the 182nd Meeting of the OP...,0.4,0.0,0.4
4,2021-11-30,OPEC marks the 5th anniversary of the landmark...,0.5,0.0,0.5
...,...,...,...,...,...
127,2024-03-19,The Seventh High-Level Meeting of the OPEC-Chi...,0.5,0.5,0.5
128,2024-03-03,Several OPEC+ countries announce extension of ...,0.7,-0.6,-0.5
129,2024-02-19,OPEC and Africa exchange views on array of key...,0.1,0.5,0.0
130,2024-02-01,52nd Meeting of the Joint Ministerial Monitori...,0.5,0.0,0.5
